In [ ]:
import os
import re
import cv2
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Try to import tensorflow_addons, if not available, define alternative metrics
try:
    import tensorflow_addons as tfa
    HAS_TFA = True
except ImportError:
    print("Warning: tensorflow_addons not available. Using alternative F1 score implementation.")
    HAS_TFA = False
    
    # Define a simple F1 score metric as alternative
    class F1Score(tf.keras.metrics.Metric):
        def __init__(self, num_classes, average='weighted', name='f1', **kwargs):
            super(F1Score, self).__init__(name=name, **kwargs)
            self.num_classes = num_classes
            self.average = average
            self.precision = tf.keras.metrics.Precision()
            self.recall = tf.keras.metrics.Recall()
            
        def update_state(self, y_true, y_pred, sample_weight=None):
            y_pred = tf.argmax(y_pred, axis=1)
            y_true = tf.argmax(y_true, axis=1)
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)
            
        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())
            
        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

from collections import Counter
from sklearn.manifold import TSNE
from matplotlib.patches import Rectangle
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# GPU Configuration and Memory Management
print("🔧 Configuring GPU and memory settings...")

# Check GPU availability
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU is available! 🚀")
    print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")
    
    # Configure GPU memory growth to avoid allocating all memory at once
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("✅ GPU memory growth enabled")
        except RuntimeError as e:
            print(f"⚠️ GPU configuration error: {e}")
else:
    print("⚠️ GPU not available. Using CPU (will be slower)")
    print("💡 Consider using Kaggle or Google Colab for GPU access")

print("🔧 Configuration complete!\n")


## Loading Prepared Dataset


In [ ]:
# Load the ODIR dataset from Kaggle
# Dataset structure: /kaggle/input/ocular-disease-recognition-odir5k/
data_path = "/kaggle/input/ocular-disease-recognition-odir5k/"
images_path = os.path.join(data_path, "preprocessed_images")

# Load the dataset CSV file
df = pd.read_csv(os.path.join(data_path, "full_df.csv"))
print(f"Dataset loaded. Total samples: {len(df)}")

# Display basic dataset information
print("\nDataset columns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

# Check if we have preprocessed train/val/test splits
# If not available, we'll create them from the dataset
train_split_path = os.path.join(data_path, "x_train.npy")
if os.path.exists(train_split_path):
    print("\nLoading preprocessed train/val/test splits...")
    x_train = np.load(os.path.join(data_path, "x_train.npy"))
    y_train = np.load(os.path.join(data_path, "y_train.npy"))
    x_val = np.load(os.path.join(data_path, "x_val.npy"))
    y_val = np.load(os.path.join(data_path, "y_val.npy"))
    x_test = np.load(os.path.join(data_path, "x_test.npy"))
    y_test = np.load(os.path.join(data_path, "y_test.npy"))
    print("Preprocessed splits loaded successfully!")
else:
    print("\nPreprocessed splits not found. Will create them from the dataset.")
    # We'll create the splits in the next cell
    x_train, y_train, x_val, y_val, x_test, y_test = None, None, None, None, None, None


In [ ]:
# Create train/val/test splits from ODIR dataset if not preprocessed
if x_train is None:
    print("Creating train/val/test splits from ODIR dataset...")
    
    # Data preprocessing
    df["class"] = df["labels"].apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))
    
    # Filter for the 5 main classes we want to use
    target_classes = ["G", "C", "A", "H", "M"]  # Glaucoma, Cataract, AMD, Hypertension, Myopia
    df_filtered = df[df["class"].isin(target_classes)]
    print(f"Filtered dataset for 5 classes: {len(df_filtered)} samples")
    
    # Create image dataset
    def create_dataset(img_list, class_label, max_images=None):
        dataset = []
        count = 0
        
        for img in img_list:
            if max_images is not None and count >= max_images:
                break
            
            image_path = os.path.join(images_path, img)
            if not os.path.exists(image_path):
                continue
                
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))
            dataset.append([np.array(image), class_label])
            count += 1
        
        return dataset
    
    # Define class mappings
    class_short2full = {
        "G": "Glaucoma",  # "G" is mapped to "Glaucoma"
        "C": "Cataract",  # "C" is mapped to "Cataract"
        "A": "Age Related Macular Degeneration",  # "A" is mapped to "Age Related Macular Degeneration"
        "H": "Hypertension",  # "H" is mapped to "Hypertension"
        "M": "Myopia"  # "M" is mapped to "Myopia"
    }
    
    # Dictionary to map short class labels to integer class indices
    class_dict = {
        "G": 0,  # "G" is mapped to index 0
        "C": 1,  # "C" is mapped to index 1
        "A": 2,  # "A" is mapped to index 2
        "H": 3,  # "H" is mapped to index 3
        "M": 4   # "M" is mapped to index 4
    }
    
    # Build dataset for each class
    dataset = []
    for class_ in target_classes:
        img_list = df_filtered.loc[df_filtered["class"] == class_]["filename"].values
        class_label = class_dict[class_]
        class_dataset = create_dataset(img_list, class_label, max_images=None)
        dataset += class_dataset
        print(f"Class {class_short2full[class_]}: {len(class_dataset)} samples")
    
    # Shuffle and split
    random.shuffle(dataset)
    
    # Convert to arrays
    X = np.array([i[0] for i in dataset])
    y = np.array([i[1] for i in dataset])
    
    # Split the data
    from sklearn.model_selection import train_test_split
    from tensorflow.keras.utils import to_categorical
    
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)
    
    # Convert to categorical
    y_train = to_categorical(y_train, 5)
    y_val = to_categorical(y_val, 5)
    y_test = to_categorical(y_test, 5)
    
    # Assign to global variables
    x_train, x_val, x_test = X_train, X_val, X_test
    
    print(f"Dataset splits created:")
    print(f"Train: {len(x_train)} samples")
    print(f"Validation: {len(x_val)} samples")
    print(f"Test: {len(x_test)} samples")


In [ ]:
# Class mappings are now defined in the data loading cell above
# This ensures they're available when needed for data processing

# Display class information
print("Class mappings:")
for short, full in class_short2full.items():
    print(f"  {short}: {full} (index: {class_dict[short]})")


## Training Part - ResNet Architecture


In [ ]:
# Define the image size
image_size = 224

# Import necessary layers from TensorFlow Keras
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D  
from tensorflow.keras.applications import ResNet50  

# Load the ResNet50 model pre-trained on ImageNet, excluding the top layer
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

# Set all ResNet50 layers as trainable
for layer in resnet.layers:
    layer.trainable = True  # Allow training of all layers in ResNet50

# Import the Sequential API for model creation
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

# Initialize the Sequential model
model = Sequential()
# Add the ResNet50 model as a feature extractor (without the top layers)
model.add(resnet)

# Add Dropout layer to prevent overfitting (rate of 0.5)
model.add(Dropout(0.5))
# Add Global Average Pooling layer to reduce dimensionality of the output from ResNet50
model.add(GlobalAveragePooling2D())

# Flatten the pooled features for dense layer processing
model.add(Flatten())

# Add BatchNormalization to standardize activations and improve training speed
model.add(tf.keras.layers.BatchNormalization())

# Add a dense layer with 512 neurons and ReLU activation
model.add(Dense(512, activation="relu"))
# Add another dense layer with 256 neurons and ReLU activation
model.add(Dense(256, activation="relu"))
# Add a dense layer with 128 neurons and ReLU activation
model.add(Dense(128, activation="relu"))
# Add the final output layer with 5 neurons for 5 classes, using softmax for multi-class classification
model.add(Dense(5, activation="softmax"))
# Display the summary of the model architecture
model.summary()


In [ ]:
# Define the number of epochs for training
epochs = 50

# Import necessary callback modules from TensorFlow Keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Reduce learning rate on plateau callback: reduces LR when the validation loss plateaus
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.5,  # Multiplies the learning rate by this factor when activated
    patience=15,  # Number of epochs to wait for improvement before reducing LR
    verbose=1,  # Print messages when learning rate is reduced
    min_delta=0.0001,  # Minimum change to qualify as an improvement
    cooldown=0,  # Number of epochs to wait before resuming normal learning rate
    min_lr=1e-7,  # Minimum learning rate, prevents LR from going below this value
)

# EarlyStopping callback: stops training if validation loss doesn't improve for 'patience' epochs
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=epochs // 5,  # Number of epochs without improvement before stopping
    restore_best_weights=True,  # Restore model weights from the epoch with the best performance
    verbose=1,  # Print messages when early stopping is triggered
)

# ModelCheckpoint callback: saves the best model during training based on validation loss
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("/kaggle/working/resnet_ocular_disease_model.h5", save_best_only=True)

# List of callbacks to be used during model training
callbacks = [checkpoint_cb, early_stopping_cb, reduce_lr]


In [ ]:
# Define the metrics to be tracked during training
METRICS = [
    tf.keras.metrics.AUC(name="auc"),  # Area under the ROC curve metric
    tf.keras.metrics.CategoricalAccuracy(name="acc"),  # Accuracy for categorical classification
    tf.keras.metrics.AUC(name="prc", curve="PR"),  # Area under the Precision-Recall curve
]

# Add F1 score metric based on availability
if HAS_TFA:
    METRICS.append(tfa.metrics.F1Score(num_classes=5, average="weighted", name="f1"))
    print("Using tensorflow_addons F1Score")
else:
    METRICS.append(F1Score(num_classes=5, average="weighted", name="f1"))
    print("Using custom F1Score implementation")

# Compile the model with Adam optimizer, categorical crossentropy loss, and the specified metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Adam optimizer with a learning rate
    loss="categorical_crossentropy",  # Loss function for multi-class classification
    metrics=METRICS  # List of metrics to evaluate during training
)


In [ ]:
# Display training information
print("Starting ResNet50 training...")
print(f"Training samples: {len(x_train)}")
print(f"Validation samples: {len(x_val)}")
print(f"Test samples: {len(x_test)}")
print(f"Number of classes: 5")
print(f"Epochs: {epochs}")
print(f"Batch size: 32")
print(f"Model will be saved as: resnet_ocular_disease_model.h5")
print("="*50)

# Train the model with callbacks
history = model.fit(x_train, y_train, batch_size=32, epochs=epochs, validation_data=(x_val, y_val), callbacks=callbacks)

print("="*50)
print("Training completed!")
print(f"Final training accuracy: {history.history['acc'][-1]:.4f}")
print(f"Final validation accuracy: {history.history['val_acc'][-1]:.4f}")
print(f"Final training loss: {history.history['loss'][-1]:.4f}")
print(f"Final validation loss: {history.history['val_loss'][-1]:.4f}")
